In [1]:
import urllib3
import pandas as pd
from bs4 import BeautifulSoup

# 1. Download Wiki HTML Page, Parse data and Create DataFrame

In [2]:
def getPreppedDataFrame(htmldata_url):
    http = urllib3.PoolManager()
    response = http.request('GET', htmldata_url)
    page = response.data
    
    # create lists
    columns = []
    postcodes = []
    boroughs = []
    neighbourhoods = []

    # use BeautifulSoup to parse HTML page and extract table/rows/columns
    soup = BeautifulSoup(page, 'html.parser')
    table = soup.find("table", class_="wikitable sortable")
    headers = table.find_all("th")
    rows = table.find_all("tr")

    # extract column headers
    for header in headers:
        columns.append(header.text.rstrip())

    # extract postcodes, boroughs and neighbourhoods from rows/columns
    for i in range(1, len(rows)):
        cols = rows[i].find_all("td")
        postcodes.append(cols[0].text.rstrip())
        boroughs.append(cols[1].text.rstrip())
        neighbourhoods.append(cols[2].text.rstrip())

    # use zip to create dataframe
    list_of_tuples = list(zip(postcodes, boroughs, neighbourhoods))  
    df = pd.DataFrame(list_of_tuples, columns=columns)  
    
    # handle invalid values for Borough
    df_clean = df.drop(df[df["Borough"] == 'Not assigned'].index)
    df_clean.reset_index(drop=True)

    # handle invalid values for Neighbourhood - no rows
    #print(df_clean[df_clean["Neighbourhood"] == 'Not assigned'])

    # group by Postcode
    df_grp = df_clean.groupby("Postcode")

    # iterate through each group to extract required data
    postcodes = []
    boroughs = []
    neighbourhoods = []
    for key,group_df in df_grp:
        #print(f"key={key}, borough={group_df['Borough'].unique()[0]}, neighbourhoods={group_df[['Neighbourhood']].apply(lambda n: ','.join(n))[0]}")
        postcodes.append(key)
        boroughs.append(group_df['Borough'].unique()[0])
        neighbourhoods.append(group_df[['Neighbourhood']].apply(lambda n: ','.join(n))[0])

    list_of_tuples = list(zip(postcodes, boroughs, neighbourhoods))  
    df_final = pd.DataFrame(list_of_tuples, columns=columns)  
    return df_final

# 2. Download Postcode coordinates file (csv) and Create DataFrame

#### Note: Geocoder packager is highly unpredictable/unreliable, so we will rely on the CSV file

In [3]:
!wget -q -O postcode_coordinates.csv https://cocl.us/Geospatial_data

In [4]:
df_coords = pd.read_csv("postcode_coordinates.csv")
df_coords.set_index("Postal Code", inplace = True) 
#df_coords
#df_coords.loc["M9W", ["Latitude", "Longitude"]][1]

# 3. Create final DataFrame containing coordinates for each Postcode

In [5]:
# get previously created dataframe
df = getPreppedDataFrame('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

latitudes = []
longitudes = []
for ind in df.index: 
    postcode = df['Postcode'][ind]
    row = df_coords.loc[postcode, ["Latitude", "Longitude"]]
    latitudes.append(row[0])
    longitudes.append(row[1])

df["Latitude"] = latitudes
df["Longitude"] = longitudes
df

C:\Users\sgirish\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437
